In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
import pandas as pd
import time
from datetime import datetime

In [8]:
df = pd.DataFrame()

In [30]:
def login(driver, url, username, password):
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//a[@href="/Market"]'))).click()
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'stds-input'))).send_keys(username)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '/html/body/div/div/div/div[2]/div/div/div/div[1]/div/fieldset/div[2]/div/div/input'))).send_keys(password)
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div/div/div/div[2]/div/div/div/div[1]/div/fieldset/div[4]/button'))).click()
    
    
def extract_data(driver, count, living):
    #시세 정보 화면 들어가기

    change = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="tbodyItemList"]/tr[' + str(count+1) + ']/td[5]/button'))
    )
    change.click()
    time.sleep(2)

    # 아이템 판매 날짜

    selling_dates = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH,  '//*[@id="weekInfo"]/tr[2]/td[1]/div')))
    selling_date = selling_dates[0].text

    # 아이템 이름
    item_names = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH,  '//*[@id="itemStats"]/span[2]')))
    item_name = item_names[0].text

    # 아이템 가격
    item_prices = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH,  '//*[@id="weekInfo"]/tr[2]/td[2]/div')))
    item_price = item_prices[0].text

    # 아이템 판매량
    sales = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH,  '/html/body/div[3]/div/div/div/div[3]/div[2]/table/tbody/tr[2]/td[3]/div')))
    sales = sales[0].text

    # 추출한 텍스트 저장
    item_data = {
        '아이템': item_name,
        '판매 날짜' : selling_date,
        '가격': item_price,
        '판매량' : sales
    }

    # 원소 추가
    sub_living = pd.DataFrame(item_data, index = [0])

    living = pd.concat([living, sub_living], ignore_index= True)

    # 화면 나가기
    exit = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="modal-deal-price"]/div/div/button'))
    )
    exit.click()
    time.sleep(1)

    return(living)

def sort_page(driver, xpath):
    link_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, xpath))
    )
    driver.execute_script("arguments[0].click();", link_element)

    time.sleep(2)

    grade_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="itemList"]/thead/tr/th[1]/a'))
    )
    driver.execute_script("arguments[0].click();", grade_element)

    time.sleep(2)
    
def made_data(driver, living):
    # 페이지 수에 있는 아이템 수 세기
    counts = driver.find_elements_by_xpath('//*[@id="tbodyItemList"]/tr')
    counts = len(counts)

    for count in range(counts):
        living = extract_data(driver, count, living)

    # 마지막 페이지 번호 뽑아내기
    onclick_value = driver.find_element(By.XPATH, '//*[@id="marketList"]/div[2]/a[7]').get_attribute('onclick')
    number = onclick_value.split('page(')[1].split(');')[0]
    last_page_num = int(number)

    for page_num in range(2, last_page_num+1):

        #페이지 변환
        change = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, f'//a[@onclick="paging.page('+str(page_num)+');"]'))
        )
        change.click()
        time.sleep(2)

        # 페이지 수에 있는 아이템 수 세기
        counts = driver.find_elements_by_xpath('//*[@id="tbodyItemList"]/tr')
        counts = len(counts)

        for count in range(counts):
            living = extract_data(driver, count, living)
            
    return(living)


# 생활 재료

In [31]:
driver = webdriver.Chrome('chromedriver')  # Chrome 드라이버 경로 설정

# Lost Ark 홈페이지 접속 & 로그인
login(driver, 'https://lostark.game.onstove.com/Main', 'kjwon4705@naver.com', 'elabest0618@')

# 아이템 가격 화면 열기
sort_page(driver, '//*[@id="lostark-wrapper"]/div/main/div/div[3]/div[1]/ul/li[8]/ul/li[1]/a')

# 데이터 구조 만들기
living = pd.DataFrame(columns = ['아이템', '판매 날짜', '가격', '판매량'])

# 페이지 수에 있는 아이템 수 세기
living = made_data(driver, living)

In [32]:
living

,아이템,판매 날짜,가격,판매량
0,철광석,2023-05-29,13.8,"24,550"
1,투박한 버섯,2023-05-29,40.5,"44,608"
2,고대 유물,2023-05-29,50.7,"327,165"
3,목재,2023-05-29,49.7,"26,382"
4,두툼한 생고기,2023-05-29,76.4,"18,088"
5,들꽃,2023-05-29,103.0,"101,718"
6,생선,2023-05-29,160.5,"117,982"
7,묵직한 철광석,2023-05-29,2.1,"105,945"
8,싱싱한 버섯,2023-05-29,4.3,"321,107"
9,수줍은 들꽃,2023-05-29,4.4,"672,163"


# 강화 재료

In [33]:
driver = webdriver.Chrome('chromedriver')  # Chrome 드라이버 경로 설정

# Lost Ark 홈페이지 접속
driver.get('https://lostark.game.onstove.com/Main')

# 거래소 버튼을 클릭하기 위해 해당 요소를 기다림
exchange_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//a[@href="/Market"]'))
)

# 거래소 버튼 클릭
exchange_button.click()


# Lost Ark 홈페이지 접속 & 로그인
login(driver, 'https://lostark.game.onstove.com/Main', 'kjwon4705@naver.com', 'elabest0618@')

# 아이템 가격 화면 열기
sort_page(driver, '//*[@id="lostark-wrapper"]/div/main/div/div[3]/div[1]/ul/li[5]/ul/li[1]/a')

#3티어 이상 조건 설정
link_element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="lostark-wrapper"]/div/main/div/div[3]/div[2]/form/fieldset/div/div[3]/div[1]/div/div[2]/label[4]'))
)
driver.execute_script("arguments[0].click();", link_element)
time.sleep(1)

find_button = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="lostark-wrapper"]/div/main/div/div[3]/div[2]/form/fieldset/div/div[4]/button[1]'))
)
driver.execute_script("arguments[0].click();", find_button)
time.sleep(1)

# 데이터 구조 만들기
reinforce = pd.DataFrame(columns = ['아이템', '판매 날짜', '가격', '판매량'])

# 페이지 수에 있는 아이템 수 세기
reinforce = made_data(driver, reinforce)


In [34]:
reinforce

,아이템,판매 날짜,가격,판매량
0,수호석 결정,2023-05-29,1.0,"1,827,348"
1,수호강석,2023-05-29,1.0,"1,077,913"
2,재봉술 : 수선 응용,2023-05-29,2.1,"23,048"
3,파괴석 결정,2023-05-29,2.0,"1,720,954"
4,정제된 수호강석,2023-05-29,2.4,"2,225,524"
5,파괴강석,2023-05-29,5.5,"3,002,523"
6,재봉술 : 수선 기본,2023-05-29,6.2,"5,752"
7,야금술 : 단조 응용,2023-05-29,5.8,"6,622"
8,야금술 : 단조 기본,2023-05-29,9.0,"1,421"
9,위대한 명예의 돌파석,2023-05-29,11.4,"1,301,347"


# 각인서

In [168]:

driver = webdriver.Chrome('chromedriver')  # Chrome 드라이버 경로 설정

# Lost Ark 홈페이지 접속
driver.get('https://lostark.game.onstove.com/Main')

# 거래소 버튼을 클릭하기 위해 해당 요소를 기다림
exchange_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//a[@href="/Market"]'))
)

# 거래소 버튼 클릭
exchange_button.click()


# Lost Ark 홈페이지 접속 & 로그인
login(driver, 'https://lostark.game.onstove.com/Main', 'kjwon4705@naver.com', 'elabest0618@')

# 아이템 가격 화면 열기
sort_page(driver, '//*[@id="lostark-wrapper"]/div/main/div/div[3]/div[1]/ul/li[4]/ul/li/a')


# 데이터 구조 만들기
recipe = pd.DataFrame(columns = ['아이템', '판매 날짜', '등급' ,'가격', '판매량'])

# 페이지 수에 있는 아이템 수 세기
counts = driver.find_elements_by_xpath('//*[@id="tbodyItemList"]/tr')
counts = len(counts)

for count in range(counts):
    
    #시세 정보 화면 들어가기

    change = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="tbodyItemList"]/tr[' + str(count+1) + ']/td[5]/button'))
    )
    change.click()
    time.sleep(2)

    # 아이템 판매 날짜

    selling_dates = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH,  '//*[@id="weekInfo"]/tr[2]/td[1]/div')))
    selling_date = selling_dates[0].text

    # 아이템 이름
    item_names = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH,  '//*[@id="itemStats"]/span[2]')))
    item_name = item_names[0].text
    
    # 아이템 등급
    item_grades = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH,  '/html/body/div[3]/div/div/div/div[1]')))
    item_grade = [item.get_attribute('data-grade') for item in item_grades]

    # 아이템 가격
    item_prices = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH,  '//*[@id="weekInfo"]/tr[2]/td[2]/div')))
    item_price = item_prices[0].text

    # 아이템 판매량
    sales = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH,  '/html/body/div[3]/div/div/div/div[3]/div[2]/table/tbody/tr[2]/td[3]/div')))
    sales = sales[0].text

    # 추출한 텍스트 저장
    item_data = {
        '아이템': item_name,
        '판매 날짜' : selling_date,
        '등급' : item_grade,
        '가격': item_price,
        '판매량' : sales
    }
    
    # 원소 추가
    sub_recipe = pd.DataFrame(item_data, index = [0])

    recipe = pd.concat([recipe, sub_recipe], ignore_index= True)
    
    # 화면 나가기
    exit = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="modal-deal-price"]/div/div/button'))
    )
    exit.click()
    time.sleep(1)
    
# 마지막 페이지 번호 뽑아내기
onclick_value = driver.find_element(By.XPATH, '//*[@id="marketList"]/div[2]/a[13]').get_attribute('onclick')
number = onclick_value.split('page(')[1].split(');')[0]
last_page_num = int(number)

for page_num in range(2, last_page_num+1):
    
    #페이지 변환
    change = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, f'//a[@onclick="paging.page('+str(page_num)+');"]'))
    )
    change.click()
    time.sleep(2)
    
    # 페이지 수에 있는 아이템 수 세기
    counts = driver.find_elements_by_xpath('//*[@id="tbodyItemList"]/tr')
    counts = len(counts)

    for count in range(counts):

        #시세 정보 화면 들어가기

        change = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="tbodyItemList"]/tr[' + str(count+1) + ']/td[5]/button'))
        )
        change.click()
        time.sleep(2)

        # 아이템 판매 날짜

        selling_dates = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH,  '//*[@id="weekInfo"]/tr[2]/td[1]/div')))
        selling_date = selling_dates[0].text

        # 아이템 이름
        item_names = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH,  '//*[@id="itemStats"]/span[2]')))
        item_name = item_names[0].text
        
        # 아이템 등급
        item_grades = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH,  '/html/body/div[3]/div/div/div/div[1]')))
        item_grade = [item.get_attribute('data-grade') for item in item_grades]

        # 아이템 가격
        item_prices = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH,  '//*[@id="weekInfo"]/tr[2]/td[2]/div')))
        item_price = item_prices[0].text

        # 아이템 판매량
        sales = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH,  '/html/body/div[3]/div/div/div/div[3]/div[2]/table/tbody/tr[2]/td[3]/div')))
        sales = sales[0].text

        # 추출한 텍스트 저장
        item_data = {
            '아이템': item_name,
            '판매 날짜' : selling_date,
            '등급' : item_grade,
            '가격': item_price,
            '판매량' : sales
        }
        sub_recipe = pd.DataFrame(item_data, index = [0])

        recipe = pd.concat([recipe, sub_recipe], ignore_index= True)

        # 화면 나가기
        exit = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="modal-deal-price"]/div/div/button'))
        )
        exit.click()
        time.sleep(1)



In [181]:
recipe['등급'] = recipe['등급'].replace('1','고급')
recipe['등급'] = recipe['등급'].replace('2','희귀')
recipe['등급'] = recipe['등급'].replace('3','영웅')
recipe['등급'] = recipe['등급'].replace('4','전설')

In [209]:
recipe_index = '[' + recipe['등급'] + '] ' + recipe['아이템']
recipe = recipe.rename(index = recipe_index)
recipe = recipe.loc[: , ['판매 날짜', '가격', '판매량']]

In [210]:
recipe

,판매 날짜,가격,판매량
[고급] 구슬동자 각인서,2023-05-28,1.0,77
[고급] 원한 각인서,2023-05-28,1.2,493
[고급] 부러진 뼈 각인서,2023-05-28,2.3,306
[고급] 폭발물 전문가 각인서,2023-05-28,22.9,187
[고급] 각성 각인서,2023-05-28,14.4,216
...,...,...,...
[전설] [슬레이어] 처단자 각인서,2023-05-28,"2,958.7","1,574"
[전설] [도화가] 만개 각인서,2023-05-28,"3,988.5","1,541"
[전설] [슬레이어] 포식자 각인서,2023-05-28,"4,198.2","1,775"
[전설] [기상술사] 질풍노도 각인서,2023-05-28,"4,271.5","1,703"


In [211]:
df = pd.concat([living, reinforce, recipe], axis = 0)
df

,판매 날짜,가격,판매량
철광석,2023-05-28,14.4,"24,560"
투박한 버섯,2023-05-28,43.3,"42,801"
목재,2023-05-28,50.6,"26,052"
고대 유물,2023-05-28,49.9,"271,268"
두툼한 생고기,2023-05-28,75.6,"17,901"
...,...,...,...
[전설] [슬레이어] 처단자 각인서,2023-05-28,"2,958.7","1,574"
[전설] [도화가] 만개 각인서,2023-05-28,"3,988.5","1,541"
[전설] [슬레이어] 포식자 각인서,2023-05-28,"4,198.2","1,775"
[전설] [기상술사] 질풍노도 각인서,2023-05-28,"4,271.5","1,703"


In [243]:
#cost = pd.read_csv("cost.csv")
cost = pd.DataFrame()

date_str = df.iloc[1, 0]
date = datetime.strptime(date_str, '%Y-%m-%d')
date = date.strftime('%m-%d')

new_cost = df.iloc[:,1:]
new_cost = new_cost.rename(columns = {'가격' : str(date) + ' 가격', '판매량' : str(date) + ' 판매량' })

data = pd.concat([cost, new_cost])
data

,05-28 가격,05-28 판매량
철광석,14.4,"24,560"
투박한 버섯,43.3,"42,801"
목재,50.6,"26,052"
고대 유물,49.9,"271,268"
두툼한 생고기,75.6,"17,901"
...,...,...
[전설] [슬레이어] 처단자 각인서,"2,958.7","1,574"
[전설] [도화가] 만개 각인서,"3,988.5","1,541"
[전설] [슬레이어] 포식자 각인서,"4,198.2","1,775"
[전설] [기상술사] 질풍노도 각인서,"4,271.5","1,703"


In [247]:
data.to_excel('data.xlsx')